In [147]:
import datetime
import h5py
import numpy as np
import os
import time
import sys

sys.path.append('../src')
import localmodule


# Define constants
n_thresholds = 100
# The array of upward thresholds is equal to
# f(t) = 1 + threshold_multiplier * (t**threshold_exponent)
# when the parameters threshold_multiplier and threshold_exponent
# are chosen such that:
# (1)    f(threshold_range_alpha*n_thresholds) = ad_hoc_threshold
# (2)    f(n_threshold) = 2 * ad_hoc_threshold - 1
# In Old Bird, ad_hoc_threshold is equal to 1.3 for Thrush and 2.0 for Tseep.
threshold_range_alpha = 0.1
data_dir = localmodule.get_data_dir()
dataset_name = localmodule.get_dataset_name()
models_dir = localmodule.get_models_dir()
sample_rate = localmodule.get_sample_rate()
args = ["unit01", "tseep", "00:09"] #                          DISABLE
#args = sys.argv[1:]                                            ENABLE
hop_duration = 0.02 # in seconds
hop_length = int(np.round(sample_rate * hop_duration))
unit_str = args[0]
odf_str = args[1]
threshold_id_start = int(args[2][:2])
threshold_id_stop = int(args[2][-2:])
threshold_id_range = range(threshold_id_start, threshold_id_stop)


# Print header.
start_time = int(time.time())
print(str(datetime.datetime.now()) + " Start.")
print("Running Old Bird onset detection functions (Thrush and Tseep) on " +
    dataset_name + ", " + unit_str + ".")
print('h5py version: {:s}.'.format(h5py.__version__))
print('numpy version: {:s}'.format(np.__version__))
print("")


# Load onset detection function.
oldbird_name = "_".join([dataset_name, "oldbird"])
oldbird_dir = os.path.join(data_dir, oldbird_name)
odf_path = os.path.join(oldbird_dir, unit_str + ".hdf5")
odf_file = h5py.File(odf_path, "r")
odf_dataset_key = "_".join([odf_str, "odf"])
odf = odf_file[odf_dataset_key]
odf_length = len(odf)


# Define arrays of thresholds.
odf_settings_key = "_".join([odf_str, "settings"])
odf_settings = odf_file[odf_settings_key]
ad_hoc_threshold = odf_settings["ratio_threshold"].value
threshold_multiplier = 2 * (ad_hoc_threshold - 1)
threshold_exponent = \
    (np.log(ad_hoc_threshold-1) - np.log(threshold_multiplier)) /\
    np.log(threshold_range_alpha)
up_threshold_abcissa = np.arange(1, 1+n_thresholds) / n_thresholds
up_thresholds = 1 + threshold_multiplier * up_threshold_abcissa**threshold_exponent
down_thresholds = 1.0 / up_thresholds


# Create directory for Old Bird in models_dir.
model_dir = os.path.join(models_dir, "Old Bird")
os.makedirs(model_dir, exist_ok=True)
out_unit_dir = os.path.join(model_dir, unit_str)
os.makedirs(out_unit_dir, exist_ok=True)
predictions_dir = os.path.join(out_unit_dir, "predictions")
os.makedirs(predictions_dir, exist_ok=True)

2017-08-08 16:33:55.469704 Start.
Running Old Bird onset detection functions (Thrush and Tseep) on BirdVox-70k, unit01.
h5py version: 2.6.0.
numpy version: 1.13.1



In [148]:
min_clip_duration = odf_settings["min_duration"].value
min_clip_length = int(np.round(min_clip_duration * sample_rate))
max_clip_duration = odf_settings["max_duration"].value
max_clip_length = int(np.round(max_clip_duration * sample_rate)) 

threshold_id = 0                                                 # DISABLE
# for threshold_id in threshold_id_range:                           ENABLE
up_threshold = up_thresholds[threshold_id]
down_threshold = down_thresholds[threshold_id]
threshold_str = str(threshold_id).zfill(2)

start_time = int(time.time())

# Initialize variables.
clip_start = 0
clip_stop = 0
clip_mid = 0
onset_odf = 0.0
offset_odf = 0.0
clip_switch = False
t = 0

odf_length = 100000000

# Scan the whole onset detection function through time
while t < odf_length:
    # If we are not inside a clip ...
    if (not clip_switch):
        # ... wait for an onset (odf[0, t] > up_threshold).
        if (odf[0, t] > up_threshold):
            # We are at the clip onset. Store start time and value of ODF.
            clip_switch = True
            clip_start = t
            onset_odf = odf[0, t]
    # Otherwise, if we are inside a clip, wait for an offset (odf[t] < down_threshold)
    # or until the maximum clip length is reached.
    elif (odf[0, t] < down_threshold) or ((t-clip_start) == max_clip_length):
        # We are at the clip offset.
        # If odf[t] > up_threshold, we should keep the clip_switch to True,
        # but that is not what Old Bird and Vesper do. 
        # see https://github.com/HaroldMills/Vesper-Tseep-Thrush/blob/master/tests/test_old_bird_detector_redux.py#L63-L65
        clip_switch = False
        # Bound the clip length from below.
        clip_length = max(t-clip_start, min_clip_length)
        # Compute time at the middle of the clip and clip duration.
        clip_stop = clip_start + clip_length
        clip_mid = int(0.5 * (clip_start+clip_stop))
        clip_time = clip_mid / sample_rate
        clip_duration = clip_length / sample_rate
        # Also store value of ODF at offset.
        offset_odf = odf[0, clip_stop]
        # TODO export clip_time, clip_duration, onset_odf, offset_odf
        # If clip length is shorter than minimum, jump to the end of clip.
        print(clip_time, clip_duration, onset_odf, offset_odf)
        if (t-clip_start) < min_clip_length:
            t = int(np.floor(clip_stop/hop_length)) * hop_length
    t = t + hop_length
    
# If the ODF ends with an onset not followed by any offset, export clip
# by using last timestamp as offset. This is unlikely to happen in practice.
if clip_switch:
    clip_switch = False
    clip_stop = odf_length - 1
    clip_mid = int(0.5 * (clip_start+clip_stop))
    clip_time = clip_mid / sample_rate
    clip_duration = clip_length / sample_rate
    offset_odf = odf[0, clip_stop]
    # TODO export clip_time, clip_duration, onset_odf, offset_odf
    

print(str(datetime.datetime.now()) + " Finish.")
elapsed_time = time.time() - int(start_time)
elapsed_hours = int(elapsed_time / (60 * 60))
elapsed_minutes = int((elapsed_time % (60 * 60)) / 60)
elapsed_seconds = elapsed_time % 60.
elapsed_str = "{:>02}:{:>02}:{:>05.2f}".format(elapsed_hours,
                                               elapsed_minutes,
                                               elapsed_seconds)
print("Total elapsed time: " + elapsed_str + ".")

23.14 0.4 1.50069 0.79069
33.26 0.4 1.69531 1.09467
448.25 0.14 1.70893 0.624542
488.1 0.4 1.51873 1.00112
714.52 0.4 1.501 1.01868
1331.98 0.4 1.51913 0.948534
1405.06 0.4 1.50603 0.883547
1996.72 0.4 1.62444 1.12726
2059.34 0.4 1.55845 1.2436
2070.14 0.4 1.52234 0.938485
2869.18 0.4 1.51671 1.04431
2909.19 0.14 1.64623 0.586651
3025.3 0.4 1.52266 1.08133
3451.34 0.4 1.63152 1.0779
3645.94 0.4 1.50412 0.836594
2017-08-08 16:34:35.364018 Finish.
Total elapsed time: 00:00:35.36.


In [172]:
str(36000.21).rjust(8)

'36000.21'

In [193]:
["{:5.3f}".format(th) for th in down_thresholds]

['0.667',
 '0.619',
 '0.590',
 '0.569',
 '0.552',
 '0.538',
 '0.527',
 '0.517',
 '0.508',
 '0.500',
 '0.493',
 '0.486',
 '0.480',
 '0.475',
 '0.470',
 '0.465',
 '0.460',
 '0.456',
 '0.452',
 '0.448',
 '0.444',
 '0.441',
 '0.438',
 '0.434',
 '0.431',
 '0.429',
 '0.426',
 '0.423',
 '0.421',
 '0.418',
 '0.416',
 '0.413',
 '0.411',
 '0.409',
 '0.407',
 '0.405',
 '0.403',
 '0.401',
 '0.399',
 '0.397',
 '0.395',
 '0.394',
 '0.392',
 '0.390',
 '0.389',
 '0.387',
 '0.386',
 '0.384',
 '0.383',
 '0.381',
 '0.380',
 '0.378',
 '0.377',
 '0.376',
 '0.374',
 '0.373',
 '0.372',
 '0.371',
 '0.370',
 '0.368',
 '0.367',
 '0.366',
 '0.365',
 '0.364',
 '0.363',
 '0.362',
 '0.361',
 '0.360',
 '0.359',
 '0.358',
 '0.357',
 '0.356',
 '0.355',
 '0.354',
 '0.353',
 '0.352',
 '0.351',
 '0.350',
 '0.349',
 '0.348',
 '0.348',
 '0.347',
 '0.346',
 '0.345',
 '0.344',
 '0.343',
 '0.343',
 '0.342',
 '0.341',
 '0.340',
 '0.340',
 '0.339',
 '0.338',
 '0.337',
 '0.337',
 '0.336',
 '0.335',
 '0.335',
 '0.334',
 '0.333']

In [201]:
"{:5.3f}".format(clip_duration)

'0.400'